In [ ]:
import warnings

warnings.filterwarnings('ignore')

## Constructing Circuits 

In this section we will explore building quantum circuits in pytket with the `Circuit` class and introduce some useful circuit primitives.

### Basic Bell circuit

In pytket we can create an instance of the `Circuit` class and add gates sequentially. We can prepare an entangled Bell state using a Hadamard and a CX gate

$$
\begin{equation}
\text{H} = \frac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix}\,, 
\quad
CX=
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{pmatrix}
\end{equation}
$$

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

bell_circ = Circuit(2, name="Bell")
bell_circ.H(0)
bell_circ.CX(0, 1)
render_circuit_jupyter(bell_circ)

In [ ]:
from pytket.extensions.qiskit import AerStateBackend

sv_backend = AerStateBackend()
result_state = sv_backend.run_circuit(bell_circ).get_state()
print("Statevector =", result_state)

Compare with the equation below...


$$
\begin{equation}
|\psi \rangle = \frac{1}{\sqrt{2}} \big(|00 \rangle + | 11 \rangle  \big)\,, \qquad \frac{1}{\sqrt{2}} \approx 0.707...
\end{equation}
$$

In [ ]:
from pytket.extensions.qiskit import AerBackend

bell_circ.measure_all()
render_circuit_jupyter(bell_circ)

backend = AerBackend()
result = backend.run_circuit(bell_circ, n_shots=1000)
print(result.get_counts())

In [ ]:
from plotting import plot_results

plot_results(result, dark_mode=True)

### Circuit Boxes (I)

* Raw quantum gates are sometimes too low level to see whats going on
* We can wrap up sub circuits into boxes and add them to circuits (useful for subroutines).
* Oracle subroutine example below

Define a trivial oracle for Grover's algorithm: 

In [ ]:
from pytket import OpType

oracle_circ = Circuit(4, name="Oracle")

oracle_circ.X(0).X(1).X(2).X(3)
oracle_circ.add_gate(OpType.CnZ, [0, 1, 2, 3])
oracle_circ.X(0).X(1).X(2).X(3)

render_circuit_jupyter(oracle_circ)

In [ ]:
from pytket.circuit import CircBox

oracle_box = CircBox(oracle_circ)
circ = Circuit(4)
circ.H(0).H(1).H(2).H(3) # Initial superposition
circ.add_circbox(oracle_box, [0, 1, 2, 3])

render_circuit_jupyter(circ)

### Circuit Boxes (II)

* Often we will need synthesise a quantum circuit to implement a gievn unitary
* Unitary synthesis is supported by TKET (Up to 3 qubits)
* **Fact:** A two qubit unitary can be implemented with at most 3 CX gates (KAK decomposition)



In [ ]:
from pytket.circuit import Unitary2qBox
from scipy.stats import unitary_group

random_su4_unitary = unitary_group.rvs(4)

u2_box = Unitary2qBox(random_su4_unitary)

test_circ = Circuit(2)
test_circ.add_unitary2qbox(u2_box, 0, 1)

render_circuit_jupyter(test_circ)

In [ ]:
from pytket.passes import DecomposeBoxes

DecomposeBoxes().apply(test_circ) # Decompose the box in place

render_circuit_jupyter(test_circ) # view raw circuit 

### Constructing a Circuit from QASM

* OpenQASM 2.0 (Open Quantum assembly language provides a low level specification of quantum circuits)
* We Can easily build a circuit from a QASM file (And also export a circuit to QASM)
* No OpenQASM 3.0 support
* QIR support a work in progress ([see pytket-qir](https://github.com/CQCL/pytket-qir))

In [ ]:
from pytket.qasm import circuit_from_qasm

qpe_circ = circuit_from_qasm("data/qpe.qasm")
print("Total number of gates =", qpe_circ.n_gates)
print("#two qubit gates =", qpe_circ.n_2qb_gates())

render_circuit_jupyter(qpe_circ)

## Brief section on Optimisation passes

* As we have seen with the phase estimation circuits we can boost the Fidelity of our circuits by reducing the number of gates in our circuit

* Lets try optimising a circuit from quantum chemistry with the general purpose `FullPeepholeOptimise` pass

In [ ]:
jw_chem_circ = circuit_from_qasm("data/H2JordanWignerMapper.qasm")
render_circuit_jupyter(jw_chem_circ)

print("Total gate count =", jw_chem_circ.n_gates)
print("#2 qubit gates =", jw_chem_circ.n_2qb_gates())

In [ ]:
from pytket.passes import FullPeepholeOptimise

FullPeepholeOptimise().apply(jw_chem_circ)

render_circuit_jupyter(jw_chem_circ)
print("Total gate count =", jw_chem_circ.n_gates)
print("#2 qubit gates =", jw_chem_circ.n_2qb_gates())

In [ ]:
from pytket.passes import auto_rebase_pass

ibm_gateset = {OpType.X, OpType.SX, OpType.Rz, OpType.CX}
ibm_rebase = auto_rebase_pass(ibm_gateset)

ibm_rebase.apply(jw_chem_circ)

The `auto_rebase_pass` works well if converting to a well known gateset i.e. one used by a hardware provider like IBM.

Rebases to other univeral sets of 1q and 2q gates can be accomplished with `RebaseCustom`.

In [ ]:
render_circuit_jupyter(jw_chem_circ)

We can also compose our own circuit transformations with `SequencePass`.

In [ ]:
from pytket.passes import SequencePass

seq_pass = SequencePass([FullPeepholeOptimise(), ibm_rebase])

jw_chem_circ = circuit_from_qasm("data/H2JordanWignerMapper.qasm")

seq_pass.apply(jw_chem_circ)

render_circuit_jupyter(jw_chem_circ)

### Working with Qiskit, Cirq and other libraries

* TKET supports easy conversion to and from qiskit circuits
* Also supported for (Cirq, pennylane, pyquil and Q# through the extensions)
* **Benefit:** Allows TKET compilation features to be used in conjunction with other tools 

In [ ]:
render_circuit_jupyter(circ)

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit #, qiskit_to_tk

qiskit_qc = tk_to_qiskit(circ) 
# use qiskit_to_tk for conversion to TKET

qiskit_qc.draw('mpl', fold=-1)

## New Features!

## Arbitrary State Prepartation

We can prepare an arbitary quantum state in pytket. Note that the size of the circuit will scale exponentailly with the number of qubits

Lets prepare the following state

$$
\begin{equation}
    |W\rangle = \frac{1}{\sqrt{3}} \big(|001\rangle + |010\rangle +  |100\rangle \big)
\end{equation}
$$

In [ ]:
from pytket.circuit import StatePreparationBox
import numpy as np

werner_state = 1 / np.sqrt(3) * np.array([0, 1, 1, 0, 1, 0, 0, 0])

werner_state_box = StatePreparationBox(werner_state)

state_circ = Circuit(3)
state_circ.add_gate(werner_state_box, [0, 1, 2])

render_circuit_jupyter(state_circ)

## ToffoliBox

A type of `Circuit` box that can implement an arbitary permutation of computational basis states.

Can "shuffle" the amplitudes of quantum state around as desired

$$
\begin{gather}
    |001\rangle \longmapsto |111\rangle \\
    |111\rangle \longmapsto |001\rangle \\
    |100\rangle \longmapsto |000\rangle \\
    |000\rangle \longmapsto |100\rangle
\end{gather}
$$

In [ ]:
from pytket.circuit import ToffoliBox

# Specify the desired permutation of the basis states
mapping = {
    (0, 0, 1): (1, 1, 1),
    (1, 1, 1): (0, 0, 1),
    (1, 0, 0): (0, 0, 0),
    (0, 0, 0): (1, 0, 0),
}

# Define box to perform the permutation
perm_box = ToffoliBox(permutation=mapping)

$$
\begin{equation}
\frac{1}{\sqrt{3}} \big(|001\rangle +  |010\rangle +|100\rangle \big)   \longmapsto \frac{1}{\sqrt{3}} \big(|000\rangle + |010\rangle + |111\rangle \big)
\end{equation}
$$

## ZX Diagrams

We have ZX diagrams in pytket. ZX is a graphical language for reasoning about quantum circuits

ZX has been used extensively in circuit simplification.

Read the [ZX Diagrams](https://cqcl.github.io/pytket/manual/manual_zx.html#) section of the user manual.


![ZX diagram](data/ZX_diagram.png)



In [ ]:
circ = Circuit(5)
circ.CCX(0, 1, 4)
circ.CCX(2, 4, 3)
circ.CCX(0, 1, 4)

render_circuit_jupyter(circ)

In [ ]:
import graphviz as gv
from pytket.zx import circuit_to_zx

# Conversion is only defined for a subset of gate types - rebase as needed
supported gateset = {OpType.Rx, OpType.Rz, OpType.X, OpType.Z, OpType.H, OpType.CZ, OpType.CX }

auto_rebase_pass(supported gateset).apply(circ)
diag, _ = circuit_to_zx(circ)

gv.Source(diag.to_graphviz_str())

In [ ]:
from pytket.zx import Rewrite

Rewrite.to_graphlike_form().apply(diag)
Rewrite.reduce_graphlike_form().apply(diag)

# Extraction requires the diagram to use MBQC generators
Rewrite.to_MBQC_diag().apply(diag)
circ, _ = diag.to_circuit()
render_circuit_jupyter(circ)

**NOTE**: The steps above are all combined in the ``ZXGraphlikeOptimisationPass``

## Summary - What have we learned?

* Overview of TKET and the challenges of running quantum algorithms
* Constructing circuits using the primitives available in pytket
* Running basic simulations of quantum circuits
* How to target different quantum devices with pytket
* Optimising circuits 
* Using TKET alongside other quantum SDKs
* New features: ZX, StatePreparation, `ToffoliBox`


Install using the command 


<p><center> <code> pip install pytket </code> </center>

Useful links
* [API docs](https://cqcl.github.io/tket/pytket/api/)
* [Notebook examples](https://github.com/CQCL/pytket/tree/main/examples)
* [User Manual](https://cqcl.github.io/pytket/manual/manual_intro.html)

Visit the TKET repository -> https://github.com/CQCL/tket 

Open source  contributions welcome! :) 


## Any Questions?